In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
#Make the connection with an open ai
openai = OpenAI()

In [4]:
# call to a frontier model i.e. OpenAI 

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


In [6]:
# Here we look the website and share the website which going to scrape.
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        #BeautifulSoup is a package which is used to parsing wabpages
        soup = BeautifulSoup(response.content, 'html.parser')
        #Fetch the title from the webpage, remove the images or style contents and store the text contents 
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
ed = Website("https://framonde.com")
print(ed.title)
print(ed.text)

Framonde - The French Classes
Our Approach
Courses
Team
Contact
                            us
+91
                        9910138144
Learn French with
FraMonde
FraMonde (means French World) is a French Language
                        Institute,
                        established in 2023 at Noida with intention of
                        opening
                        doors
                        to a new world of opportunities, from connecting
                        with
                        French
                        speakers worldwide to exploring the culture of
                        France
                        and
                        other French-speaking countries.
ONLINE 1 to 1 CLASSES & GROUP CLASSES
established in 2023 at Noida
We are offering high quality
French
                        language
courses at affordable prices to the
                    people
                    around the
                    globe. We are very passionate about helping you ac

In [ ]:
## Types of prompts

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [20]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Framonde - The French Classes
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Our Approach
Courses
Team
Contact
                            us
+91
                        9910138144
Learn French with
FraMonde
FraMonde (means French World) is a French Language
                        Institute,
                        established in 2023 at Noida with intention of
                        opening
                        doors
                        to a new world of opportunities, from connecting
                        with
                        French
                        speakers worldwide to exploring the culture of
                        France
                        and
                        other French-speaking countries.
ONLINE 1 to 1 CLASSES & GROUP CLASSES
established in 2023 at Noida
We are offering high quality

In [ ]:
## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]


In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [17]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you must be a math wizard! The answer is 4. But don't worry, I won't tell anyone your secret!


In [21]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Framonde - The French Classes\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nOur Approach\nCourses\nTeam\nContact\r\n                            us\n+91\r\n                        9910138144\nLearn French with\nFraMonde\nFraMonde (means French World) is a French Language\r\n                        Institute,\r\n                        established in 2023 at Noida with intention of\r\n                        opening\r\n                        doors\r\n                        to a new world of opportunities, from connecting\r\n                        with\r\n                        French\r\n             

In [23]:
# call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [24]:
summarize("https://framonde.com")

'# Framonde - The French Classes\n\nFramonde, meaning "French World," is a language institute founded in 2023 in Noida, dedicated to teaching French. The institute offers **online 1-to-1 and group classes** accessible globally, allowing students to learn at their convenience without geographical limitations. \n\n## Courses Offered\nFramonde provides a variety of **French language courses** tailored for different audiences:\n- **Tuition for School Students**: Engaging classes aligned with school curricula.\n- **DELF Exam Preparation/University Test**: Specialized programs for students aiming to pursue studies in French language, literature, and culture.\n- **French for Working Professionals**: Courses designed for professionals seeking to enhance their French proficiency for work.\n\n## Our Approach\nThe coaching program is personalized, catering to learners of all levels from beginners to advanced speakers. The institute emphasizes quality learning and adapts to individual learning sty

In [25]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [27]:
display_summary("https://framonde.com")

# Framonde - The French Classes

**Overview**  
FraMonde, meaning "French World," is a French language institute established in 2023 in Noida. The institute aims to provide high-quality French language education, opening doors to new opportunities and cultural exploration for learners worldwide. 

**Courses Offered**  
- **General French Classes**: Catering to all levels from beginners to advanced speakers with a personalized coaching approach.
- **Tuition for School Students**: Classes designed to align with school curriculums, making learning interactive and enjoyable.
- **DELF Exam Preparation**: Focused courses for students aspiring to specialize in French language, literature, and culture.
- **French for Working Professionals**: Tailored classes for professionals looking to enhance their French language skills as part of their job requirements.

**Learning Format**  
All classes are conducted online, allowing flexibility for students to learn from anywhere. This is beneficial for those who might relocate or travel.

**About the Team**  
The institute is led by Ms. Joshita Batra, a passionate educator with an M.A. in French and 10 years of teaching experience in both traditional and online settings.

**Contact Information**  
Interested individuals can reach out via email or visit their registered office in Greater Noida-West.

This summary captures the core offerings and mission of Framonde, along with details about their team and learning approach. No news or announcements were noted.